# 動的メモリ管理とスマートポインタ


## キーポイント

* メモリの管理方法には「自動」「静的」「動的」の3種類がある
* 動的なメモリ管理は、必要なときだけメモリを確保し、不要になったらメモリを解放する仕組み
* `new`キーワードでメモリを確保し、`delete`キーワードでメモリを解放する
* 動的なメモリを適切に管理するのは難しいので、メモリリークなどの問題を起こしやすい
* 安全に動的メモリ管理を行うには、`shared_ptr`(シェアード・ポインタ)型を使う


----

## 1 　３種類のメモリ管理

----


### 1.1 メモリの管理方法について

これまで何度か「変数はメモ帳に書かれたメモ」という例え話をしてきました。<br>
実は、コンピュータがメモを扱う方法は、次の3種類に分けられます。

* 自動的なメモリ管理
* 静的(せいてき)なメモリ管理
* 動的(どうてき)なメモリ管理


### 1.2 自動的なメモリ管理

「自動的なメモリ管理」は、関数の中で宣言する変数の扱い方です。<br>
宣言した時点でメモ帳に書かれ、スコープが終了するとメモ帳から消されます。

この種類の変数は「ローカル変数」や「自動変数」と呼ばれます。<br>
次のプログラムで使われている変数は、すべてローカル変数です。

**コード**

```cpp
#include <iostream>
#include <string>
using namespace std;

double f(int a) { // aはローカル変数
  double x = a * 2; // xはローカル変数
  return x;
}

int main() {
  int y = f(10); // yはローカル変数
  cout << y << endl;

  string s = "123"; // sはローカル変数
  for (int i = 0; i < (int)s.size(); i++) { // iはローカル変数
    cout << s[i] << " ";
  }
}
```

同じスコープに属するローカル変数は、宣言とは逆順に削除されます。<br>
例えば、上記のプログラムの場合、同じスコープに属する`y`と`s`は、`s`→`y`の順で削除されます。


### 1.3 静的(せいてき)なメモリ管理

「静的なメモリ管理」は、関数の外で定義する、または`static`(スタティック)キーワードを付けて宣言する変数の扱い方です。<br>
プログラムの実行直後、`main`関数が始まるより前にメモが作られ、プログラムが終了するまで消えません。

この種類の変数は「スタティック変数」や「静的変数」と呼ばれます。<br>
さらに、関数の外で宣言したスタティック変数は、特別に「グローバル変数」と呼ばれます。

スタティック変数は安全に使うことが難しいため、多用してはいけません。

次のプログラムは、スタティック変数を使った例です。

**コード**

```cpp
#include <iostream>
#include <string>
using namespace std;

int x = 3; // xはグローバル変数(スタティック変数)
string s = "abc"; // sはグローバル変数(スタティック変数)

int f(int a) {
  static int y = 0; // yはスタティック変数
  y += a;
  return y;
}

int main() {
  cout << f(x) << endl;
  cout << f(5) << endl;
  cout << s << endl;
}
```

**実行結果**

```txt
3
8
abc
```

関数`f`の2回目の呼び出しで、戻り値が`5`ではなく`8`になっている点に注目してください。<br>
スタティック変数は関数が終了しても削除されないので、最後に関数を実行したときの値が残っているのです。

スタティック変数は、宣言とは逆順に削除されます。<br>
例えば、上記のプログラムのスタティック変数`x`と`s`は、`s`→`x`の順で削除されます。

>`static`(スタティック)は「長い間動かず、変化せず、その場に留まる。静的。」という意味です。<br>
>`global`(グローバル)は「全世界。世界的な」という意味です。プログラミング用語では「プログラムのどこからでも見える、使える」ことを意味します。


### 1.4 動的(どうてき)なメモリ管理

「動的なメモリ管理」では、プログラムからの指示で、好きなときにメモを作ったり消したりできます。<br>
メモがどこに作られるかはコンピュータまかせです。<br>
プログラムからは「メモの位置」、つまりポインタや参照を使ってメモを読み書きします。

動的なメモリ管理で作成されたデータは、「オブジェクト」などと呼ばれます。

動的なメモリ管理では、`new`キーワードや`delete`キーワードを使って確保したメモリに、データを保存します。<br>
データの読み書きには、メモリアドレスを記録したポインタを使います。



```cpp
#include <iostream>
#include <string>
using namespace std;

int* f(int a) {
  int* p = new int(a); // int型のオブジェクトを作成
  return p; // オブジェクトのアドレスを返す
}

int main() {
  int* x = f(1); // オブジェクトのアドレスを受け取る
  string* y = new string("23"); // string型のオブジェクトを作成

  cout << *x << *y << endl;

  delete y; // yが指すアドレスにあるオブジェクトを削除
  delete x; // xが指すアドレスにあるオブジェクトを削除
}
```

**実行結果**

```txt
123
```

`new`によって作成されたデータは、`delete`されるまで消えることはありません。<br>
そのため、スコープを超えてデータをやり取りできます。

自動的なメモリ管理や静的なメモリ管理では、データの削除はコンピュータが管理しました。<br>
ですが、動的なメモリ管理では、プログラマが責任を持ってデータを削除(`delete`を使うなど)しなくてはなりません。


----

## 2 安全でない動的メモリ管理

----


### 2.1 動的なメモリ管理を使う理由

プログラムから利用できるメモリは、コンピュータに搭載されている物理的な「メモリ」そのものです。<br>
コンピュータのOSがプログラムを実行するとき、「物理的なメモリ」の一部をプログラム用に割り当てます。<br>
そして、プログラムは、割り当てられたメモリの範囲内で、必要な処理を実行します。

以下の図は、OSから割り当てられたメモリの概念図です。

$
\begin{array}{|c|c|ccc|c|}
\hline
テキスト & データ & 　　 & ヒープ & 　　 & スタック \\
\hline
\end{array}
$

* テキスト: プログラム自身が読み込まれるメモリ領域です。
* データ: 「静的な管理」用のメモリ領域です。
* ヒープ: 「動的な管理」用のメモリ領域です。
* スタック: 「自動的な管理」用のメモリ領域です。

「テキスト」と「データ」の２つは、プログラムをビルドした時点で大きさが決まります。<br>
対して、「ヒープ」と「スタック」の大きさはプログラムの実行中に変化します。

小規模なプログラムであれば、「自動的なメモリ管理」と「静的なメモリ管理」だけでも、十分に処理を完了させられます。

しかし、ある程度以上の規模を持つプログラムでは、

&emsp;**利用可能なメモリが、すべての処理を行うために必要なメモリ量より少ない**

という事態が発生します。


#### JRPGの動的なメモリ管理

例えば、2025年現在のゲーム機のメモリ容量はおよそ4GB～32GBです。<br>
しかし、ゲームアプリの容量は数百GBに達することもあり、とてもすべてをメモリに読み込むことはできません。

そこで「動的なメモリ管理」の出番です。例えば、日本の古典的なロールプレイングゲームを考えてみましょう。

1. マップ移動画面ではマップデータだけがメモリに読み込まれる。
2. 戦闘が始まるとマップデータのメモリが解放され、代わりに戦闘データがメモリに読み込まれる。
3. このとき、この戦闘で登場する味方と敵のデータだけが読み込まれ、登場しないデータは読み込まれない。
4. 戦闘が終わると戦闘データ用のメモリは解放され、再びマップデータがメモリに読み込まれる。

このように、「動的なメモリ管理」を使うと、「ある処理を実行中だけ必要なメモリを確保」し、不要になったら「他の処理のために解放」できます。この仕組みによって、少ないメモリ容量でも膨大なデータを扱うことができるのです。


#### 動的なメモリ管理のための機能

さて、肝心の「動的なメモリ管理」を行う方法ですが、C++には以下の2つの機能が組み込まれています。

* `new`(ニュー)キーワードと`delete`キーワードを使ったメモリ管理
* `malloc`(マロック)関数と`free`(フリー)関数を使ったメモリ管理

ただし、これらの機能は安全性に問題があるため、2025年現在では直接使われることは少なくなっています。<br>
しかし、あとで説明する「安全な動的メモリ管理」でも、内部的にはこれらの機能を使っています。

また、これらの機能が使われているプログラムを見たり、利用したりすることもあるでしょう。<br>
そこで、この章では、これらの基本的なメモリ管理の使いかたを解説します。


### 2.2 newとdelete

`new`(ニュー)と`delete`(デリート)は、C++言語の最も基本的な動的メモリ管理の仕組みです。

動的メモリ管理では、実際には「ヒープ」と呼ばれるメモリ領域を管理します。

プログラムが起動すると、ヒープメモリ全体が「未使用」状態に設定されます。<br>
`new`が呼ばれると、ヒープメモリの一部を「使用中」に設定し、そのアドレスを返します。<br>
`delete`が呼ばれると、アドレスで指定されたメモリを「未使用」に戻します。

`new`キーワードの書式は次のようになっています。

>**書式**
>
>```cpp
>new 型;         // 型ひとつぶんのメモリを確保し、デフォルトコンストラクタで初期化
>new 型(初期値); // 型ひとつぶんのメモリを確保し、初期値に対応するコンストラクタで初期化
>new 型[要素数]; // 型の要素数の個数の連続したメモリを確保し、デフォルトコンストラクタで初期化
>```
>
>**戻り値**
>
>最初の2つの形式では、指定された型を格納するためのメモリを確保し、型のコンストラクタを呼び出し、確保したメモリのアドレスを返します。
>
>`new 型[要素数]`の形式では、「要素数」で指定されたサイズの配列を格納するメモリを確保し、すべての要素にコンストラクタを呼び出して、配列の先頭アドレスを返します。この形式では初期値を指定できません。
>
>どの形式でも、戻り値の型は「指定した型のポインタ型」となります(例: `new int`の場合`int*`)。

`delete`キーワードの書式は次のようになっています。

>**書式**
>
>```cpp
>delete ポインタ変数;   // ひとつぶんのメモリを解放する
>delete[] ポインタ変数; // 配列のメモリを解放する
>```
>
>**戻り値**
>
>ポインタ変数が指すオブジェクトのデストラクタを実行し、メモリを開放して他のプログラムから使えるようにします。<br>
>`delete[]`形式は配列用です。配列ではない変数に`delete[]`を使ったり、逆に配列変数に`delete`を使ってしまうと、論理エラーや実行時エラーが起こります。

次のプログラムは、`new`と`delete`の使用例です。

**コード**

```cpp
#include <iostream>
using namespace std;

// 敵の構造体
struct Enemy {
  Enemy(int h, int p, int a) : hp(h), power(p), armor(a) {
    cout << "コンストラクタを実行" << endl; }

  ~Enemy() { cout << "デストラクタを実行" << endl; }

  int hp, power, armor;
};

// 敵を作成する関数
Enemy* CreateEnemy(int h, int p, int a) {
  return new Enemy(h, p, a); // Enemy用のメモリを確保し、Enemyオブジェクトを初期化
}

// メイン関数
int main() {
  Enemy* p = CreateEnemy(6, 4, 3); // Enemyオブジェクトを作成

  cout << p->hp << ' ' << p->power << ' ' << p->armor << endl;

  delete p; // Enemyオブジェクトを削除し、メモリを解放
}
```

**実行結果**

```txt
コンストラクタを実行
6 4 3
デストラクタを実行
```

次のプログラムは、配列版の`new`と`delete[]`の使用例です。

**コード**

```cpp
#include <iostream>
#include <algorithm>
using namespace std;

// メイン関数
int main() {
  int* p = int[10]; // 配列のメモリを確保

  for (int i = 0; i < n; i++) {
    cin >> p[i];
  }

  sort(p, p + 10);

  for (int i = 0; i < n; i++) {
    cout << p[i] << ' ';
  }
  cout << endl;

  delete[] p; // 配列のメモリを解放
}
```

**入力データ**

```txt
5
3 1 4 1 5
```

**実行結果**

```txt
1 1 3 4 5
```


#### メモリ・リーク

`new`と`delete`をつかうとき、最も重要なことは「必ず`delete`を実行する」ことです。<br>
動的な管理で作成されたオブジェクトは、`delete`を実行しない限り削除されないからです。もし削除し忘れると、オブジェクトが占めていたメモリ領域は「使用中」のままになり、プログラムが終了するまで二度と使えなくなってしまいます。

この状態は「メモリ・リーク(メモリ漏れ)」と呼ばれています。

コンピュータに搭載されているメモリは有限なので、メモリ・リークが頻発すると、コンピュータ全体で使用できるメモリがどんどん減っていきます。
その結果、徐々にコンピュータの実行効率が低下し、最終的には実行時エラーを起こしてアプリが強制終了します。

また、ポインタは「どこかの変数を参照している」状態と、「`new`で作られたオブジェクトを参照している」状態を区別できません。

```cpp
int a = 1;
int* b = &a;         // 変数aを参照している
int* c = new int(2); // 動的な管理で作られたオブジェクトを参照している

// 型が同じなら参照先を交換できる
int* tmp = b;
b = c;
c = tmp;
// この時点で、ポインタbとcは、それぞれ何を参照している？
```

そのため、プログラマが「ここのポインタXは`new`で作ったやつを参照してるから`delete`する。あっちのポインタYは変数を参照してるから`delete`しない。」という判断を下さなくてはなりません。<br>
もし判断を間違えると「メモリ・リーク」が起こります。


#### deleteしたメモリを読み書きしない

`delete`によって削除したあとのオブジェクトを読み書きしないように、注意する必要があります。<br>
もし削除したオブジェクトを読み書きしようとすると「実行時エラー」や「論理エラー」が発生します。

**コード**

```cpp
#include <iostream>
using namespace std;

struct Enemy {
  Enemy(int h, int p, int a) : hp(h), power(p), armor(a) { cout << "コンストラクタを実行" << endl; }
  ~Enemy() { cout << "デストラクタを実行" << endl; }

  int hp, power, armor;
};

int main() {
  Enemy* p = new Enemy(6, 4, 3); // Enemyオブジェクトを作成

  cout << p->hp << ' ' << p->power << ' ' << p->armor << endl;

  delete p; // Enemyオブジェクトを削除

  // 削除したオブジェクトのデータを読もうとすると…
  cout << p->hp << ' ' << p->power << ' ' << p->armor << endl;
}
```

**実行結果**

```txt
コンストラクタを実行
6 4 3
デストラクタを実行
83096 0 -1910714224
```


#### 多重deleteをしない

同じオブジェクトに`delete`を2回以上実行してしまうと、2回目の`delete`で「実行時エラー」が発生します。

**コード**

```cpp
#include <iostream>
using namespace std;

struct Enemy {
  Enemy(int h, int p, int a) : hp(h), power(p), armor(a) { cout << "コンストラクタを実行" << endl; }
  ~Enemy() { cout << "デストラクタを実行" << endl; }

  int hp, power, armor;
};

int main() {
  Enemy* p = new Enemy(6, 4, 3); // Enemyオブジェクトを作成

  cout << p->hp << ' ' << p->power << ' ' << p->armor << endl;

  delete p; // Enemyオブジェクトを削除
  delete p; // 削除済みのオブジェクトを再び削除しようとすると…
}
```

**実行結果**

```txt
コンストラクタを実行
6 4 3
デストラクタを実行
デストラクタを実行
free(): double free detected in tcache 2
```

多重`delete`問題に限れば、ポインタ変数に`nullptr`を代入することで回避できます(`delete`は`nullptr`に対して何もしないため)。<br>
ただし、`nullptr`の代入で防げるのは多重`delete`だけです。他の問題は防げません。

**コード**

```cpp
#include <iostream>
using namespace std;

struct Enemy {
  Enemy(int h, int p, int a) : hp(h), power(p), armor(a) { cout << "コンストラクタを実行" << endl; }
  ~Enemy() { cout << "デストラクタを実行" << endl; }

  int hp, power, armor;
};

int main() {
  Enemy* p = new Enemy(6, 4, 3); // Enemyオブジェクトを作成

  cout << p->hp << ' ' << p->power << ' ' << p->armor << endl;

  delete p; // Enemyオブジェクトを削除
  p = nullptr; // 削除後に nullptr を代入
  delete p; // p は nullptr なので何も起こらない
}
```

**実行結果**

```txt
コンストラクタを実行
6 4 3
デストラクタを実行
```


### 2.3 vectorと動的メモリ管理

動的メモリ管理で作成したポインタを、`vector`配列に設定する場合、データを削除する前に`delete`を実行しなくてはなりません。

**コード**

```cpp
#include <iostream>
#include <vector>
using namespace std;

int main() {
  vector<int*> v(5); // int*の配列

  // newを使ってメモリを確保
  for (int i = 0; i < 5; i++) {
    v[i] = new int(i + 1);
  }

  for (auto i = v.begin(); i != v.end(); i++) {
    cout << **i << " "; // イテレータとポインタで、'*'記号が2つ必要
  }
  cout << endl;

  //v.pop_back(); // NG: deleteしていない
  //v.claer(); // NG: deleteしていない

  // OK. 末尾の要素をdeleteしてから削除
  delete v.back();
  v.pop_back();

  // OK. すべての要素をdeleteしてから削除
  for (auto i = v.begin(); i != v.end(); i++) {
    delete *i;
  }
  v.clear();
}
```

**実行結果**

```txt
1 2 3 4 5
```

`delete`せずにポインタ変数を削除してしまうと、メモリ・リークが起こります。<br>
これは汎用アルゴリズムでも同様です。`remove`関数など、削除が起こりうる関数はすべて使えないと考えてください。


### 2.4 mallocとfree

`malloc`(マロック)関数と`free`(フリー)関数は、C言語の動的メモリ管理の仕組みです。<br>
C++ではほとんど使われないため、最低限の解説だけを行います。

`malloc`関数の書式は次のようになっています。

>**書式**
>
>```cpp
>void* malloc(size_t size);
>```
>
>**引数**
>
>* size&emsp;確保するメモリのバイト数
>
>**戻り値**
>
>少なくとも`size`バイトのメモリ領域を確保し、確保したメモリ領域の先頭アドレスを返します。<br>
>戻り値の型は`void*`なので、必要な型にキャストして使います。

`free`関数の書式は次のようになっています。

>**書式**
>
>```cpp
>void free(void* p);
>```
>
>**引数**
>
>* p&emsp;解放するメモリアドレス
>
> 引数`p`が指すアドレスのメモリ領域を解放し、他のプログラムから使えるようにします。

`new`と`delete`と比較したとき、最大の違いは「コンストラクタもデストラクタも実行しない」ことです。<br>
そのため、コンストラクタやデストラクタを持つ構造体やクラスを`malloc`で作成しようとすると、かなり面倒なことになります。

`int`などの組み込み型や、メンバ変数だけの構造体、クラスの場合は、`malloc`と`free`を使っても問題もありません。

**コード**

```cpp
#include <iostream>
using namespace std;

struct Enemy {
  int hp, power, armor;
};

int main() {
  Enemy* p = (Enemy*)malloc(sizeof(Enemy));
  p->hp = 6;
  p->power = 4;
  p->armor = 3;

  cout << p->hp << ' ' << p->power << ' ' << p->armor << endl;

  free(p);
}
```

**実行結果**

```txt
6 4 3
```

`malloc`関数で確保したメモリは`free`関数で解放しなくてはなりません。<br>
`new`と`delete`の場合と同様に、解放し忘れると「メモリ・リーク」が起こります。<br>
削除したオブジェクトを読み書きすると「実行時エラー」や「論理エラー」が発生するのも同様です。

また、あるポインタが`new`で作られたのか、`malloc`で作られたのかを区別する方法はありません。<br>
そのため、プログラマが「このポインタは`malloc`で作ったやつだから、`free`を使わなくちゃ」という判断を下す必要があります。<br>
もし判断を間違えると「論理エラー」や「実行時エラー」が発生します。


----

## 3 安全な動的メモリ管理

----


### 3.1 スマート・ポインタ


#### new, deleteの問題点

`new`と`delete`を使う場合、問題のひとつは「`delete`のタイミングをプログラマが適切に管理する必要がある」ということでした。<br>
そのため、うっかり必要なオブジェクトを`delete`してしまったり、些細な勘違いで`delete`を書き忘れてしまう危険性があります。

これらの問題は、コンピュータが管理する「自動的なメモリ管理」や「静的なメモリ管理」では起こりません。<br>
もし「動的なメモリ管理」でも、コンピュータに削除のタイミングを任せられるなら、問題の多くが解決されると期待できます。


#### スマートポインタで問題を解決

そこで、「スマート・ポインタ」が開発されました。<br>
スマートポインタは、オブジェクトを削除するタイミングを、コンピュータに管理させるように設計されたクラスです。<br>
そのため、プログラマは削除のタイミングを考える必要はありません。

C++には、次に示す3種類のスマート・ポインタがあります。

* `shared_ptr`(シェアード・ポインタ): 標準的なスマートポインタ
* `unique_ptr`(ユニーク・ポインタ): オブジェクトを指すポインタがひとつだけのスマートポインタ
* `weak_ptr`(ウィーク・ポインタ): 「循環参照(じゅんかん・さんしょう)」という問題を解決するためのスマートポインタ

これらのスマートポインタに共通する仕組みは、「デストラクタでメモリを解放する」というものです。<br>
デストラクタは、変数を削除するときに自動的に呼び出されるので、変数が削除されると、自動的にメモリも解放されるわけです。

この章では、基本となる`shared_ptr`だけ解説します。<br>
`unique_ptr`と`weak_ptr`については、`cpprefjp.github.io`などのリファレンスサイトを見てください。


### 3.2 shared_ptr(シェアード・ポインタ)

`shared_ptr`クラスは、C++の標準的なスマートポインタです。

`shared_ptr`を使うには`<memory>`ヘッダをインクルードします。<br>
`shared_ptr`は、テンプレートを使って次のように宣言されています。

```cpp
template<typename T>
class shared_ptr;
```

`shared_ptr`型の変数が参照しているオブジェクトは、次の場合に自動的に削除されます。

* ポインタ変数が削除される
* ポインタ変数に「別のオブジェクト」のアドレスが代入される
    * 「別のオブジェクト」を指すポインタ変数を代入
    * `make_shared`(次節で解説)で「別のオブジェクト」を作って代入
* `nullptr`を代入
* `reset`メンバ関数を呼び出す

ただし、ポインタ変数のコピーが残っている場合は削除されません。<br>
オブジェクトが削除されるのは「オブジェクトを参照するポインタ変数が、ひとつもなくなったとき」です。


>なお、削除したオブジェクトを読み書きしようとすると、実行時エラーになります。<br>
>手動でオブジェクトを削除した場合、削除後に読み書きをしないように注意してください。


#### make_shared(メイク・シェアード)関数

動的メモリ管理される「オブジェクト」を作るには、`make_shared`(メイク・シェアード)関数を使います。<br>
`make_shared`関数を使うには`<memory>`ヘッダをインクルードします。

>**書式**
>
>```cpp
>template<typename T>
>shared_ptr make_shared<T>(コンストラクタに渡す引数);
>```
>
>**テンプレート引数**
>
>T&emsp;作成する型の名前
>
>**引数**
>
>Tに指定した型のコンストラクタに渡す引数です。内容はTの型によって異なります。
>
>**戻り値**
>
>確保したメモリアドレスを管理する`shared_ptr`型の値を返します。

`make_shared`関数は、次の処理を行います。

1. 指定された「データの型」を記録するためのメモリ領域を、ヒープ領域から確保する
2. 確保したメモリ領域に対して「データの型」のコンストラクタを実行する
3. 確保したメモリを管理する`shared_ptr`型のデータを返す。

次のプログラムは、`make_shared`関数と`shared_ptr`型を使った例です。

**コード**

```cpp
#include <iostream>
#include <memory>
using namespace std;

struct Enemy {
  Enemy(int h, int p, int a) : hp(h), power(p), armor(a) { cout << "コンストラクタを実行" << endl; }
  ~Enemy() { cout << "デストラクタを実行" << endl; }

  int hp, power, armor;
};

int main() {
  shared_ptr<Enemy> p = make_shared<Enemy>(6, 4, 3);
  cout << p->hp << ' ' << p->power << ' ' << p->armor << endl;
}
```

**実行結果**

```txt
コンストラクタを実行
6 4 3
デストラクタを実行
```

このプログラムは、返された「アドレス」を`shared_ptr`変数に代入し、ポインタを使ってデータを読み書きします。<br>
作成方法こそ普通のポインタとは異なりますが、使いかたは普通のポインタと同じです。


#### make_shared関数とautoキーワード

上で書いた`make_shared`関数のプログラムでは、作成したい型を2回書いています。

```cpp
//         ↓１回                 ↓２回
shared_ptr<Enemy> p = make_shared<Enemy>(6, 4, 3);
```

もし型を変えたくなったら、この2か所を同時に書き換えなくてはなりません。<br>
ですが、このような書き換えはエラーの原因になりやすいため、できれば型は1回だけ書くようにしたいものです。<br>
そこで、`auto`(オート)キーワードを使います。

`auto`キーワードを使うと、上の例は次のように書けます。

```cpp
//                   ↓１回だけ
auto p = make_shared<Enemy>(6, 4, 3);
```

このように`auto`を使うと、型を書く回数を最小限でき、プログラムも短くなります。

>役に立つ場面では、積極的に`auto`キーワードを利用しましょう


#### 変数をコピーしたときの挙動

普通のポインタ変数をコピーすると、コピー先のポインタ変数もおなじメモリアドレスを指します。<br>
次のプログラムは、ポインタ変数のコピーの動作を示す例です。

**コード**

```cpp
#include <iostream>
using namespace std;

int main() {
  int* a = new int(10);

  int* b = a; // aとbは同じメモリアドレスを指す

  *b += 5;
  cout << *a << endl; // 15が出力される

  delete b; // OK: aとbは同じメモリアドレスを指している
            //     bを通じてメモリを解放したので、aはdeleteする必要はない
}
```

このくらい短いプログラムであれば、ポインタ変数`a`と`b`のどちらか片方を`delete`すれば、もう片方を`delete`する必要はないし、してはいけない、ということを覚えていられるでしょう。

ですが、もっと長いプログラムで、さまざまな場所にポインタ変数のコピーがある、という状況では、「正しく1度だけ`delete`する」というのは非常に困難になります。

この問題も、`shared_ptr`を使えば解決できます。`shared_ptr`は「コピーされた回数」を覚える機能を持っているからです。

次のプログラムは、`shared_ptr`を使って上のプログラムを書き直した例です。

**コード**

```cpp
#include <iostream>
#include <memory>
using namespace std;

int main() {
  shared_ptr<int> a = make_shared<int>(10);

  shared_ptr<int> b = a; // aとbは同じメモリアドレスを指す
                         // aはコピーされたことを記録する
  *b += 5;
  cout << *a << endl; // 15が出力される
} // ここでaとbが削除され、メモリの解放が1度だけ行われる
```

このように、`shared_ptr`を使うと、ポインタのコピーの管理とメモリの解放を、全てコンピュータに任せられます。<br>
どれだけコピーを作ろうと、いつ、どこに`delete`を書くべきかについて、頭を悩ませる必要はありません。


### 3.3 vectorとshared_ptr

`new`で取得したポインタを`vector`配列に格納する場合、削除する前に`delete`しなくてはならない、という説明をしました。<br>
ですが、`shared_ptr`を使えば、削除の心配はなくなります。

また、汎用アルゴリズムも使えるようになります。<br>
というのも、汎用アルゴリズムで削除された場合でも、必ずデストラクタが呼び出されるからです。

次のプログラムは、`vector`と動的メモリ管理を組み合わせたプログラムを、`shared_ptr`を使うように修正したものです。

**コード**

```cpp
#include <iostream>
#include <vector>
#include <memory>
using namespace std;

int main() {
  vector<shared_ptr<int>> v(5); // shared_ptr<int>の配列

  // make_sharedを使ってメモリを確保
  for (int i = 0; i < 5; i++) {
    v[i] = make_shared<int>(i + 1);
  }

  for (auto i = v.begin(); i != v.end(); i++) {
    cout << **i << " "; // イテレータとポインタで、'*'記号が2つ必要
  }
  cout << endl;

  v.pop_back(); // OK. 自動的にdeleteされる
  v.clear(); // OK. 自動的にdeleteされる

  // NG: shared_ptrが管理しているメモリを、勝手にdeleteしてはいけない
  //delete v.back();
  //v.pop_back();

  // NG: shared_ptrが管理しているメモリを、勝手にdeleteしてはいけない
  //for (auto i = v.begin(); i != v.end(); i++) {
  //  delete *i;
  //}
  //v.clear();
}
```

以前のプログラムと比較すると、`delete`が消えて、`pop_back`や`clear`メンバ関数でいつでも要素を削除できるようになっています。<br>
プログラムに登場してはいませんが、もちろん`erase`も使えます。

このように、`shared_ptr`を使うと、動的メモリ管理で作られたデータであることをあまり意識せずにプログラムが作れます。<br>
`int`のような組み込み型に近い感覚でプログラムを書けるわけです。


#### 「構造体のポインタの配列」と、メンバ変数の読み書きについて

次のように、構造体のポインタを配列にする場合を考えます。

```cpp
struct Point {
  int x, y;
};

vector<shared_ptr<Point>> a;
for (int i = 0; i < 5; i++) {
  auto p = make_shared<Point>();
  *p = { i + 1, i + 5 };
  a.push_back(p);
}
```

この状態で、for文を使ってメンバ変数を読み書きするには、次の2種類の方法があります。

**添字を使う場合**

```cpp
for (int i = 0; i < 5; i++) {
  cout << a[i]->x << ',' << a[i]->y << ' ';
}
```

**イテレータを使う場合**

```cpp
for (auto i = a.begin(); i != a.end(); i++) {
  cout << (*i)->x << ',' << (*i)->y << ' ';
}
```

イテレータを使う場合、`*i->x`とは書けない点に注意してください。これは、`*`演算子より`->`演算子のほうが優先順が高いためです。<br>
優先順のために`*i->x`は`*(i->x)`と解釈され、`shared_ptr`型に`x`というメンバはないので、コンパイルエラーになってしまいます。

コンパイルエラーを防ぐには、括弧を使って`(*i)->x`または`(**i).x`と書く必要があります。


### 3.4 安全な動的メモリ管理を使おう

ここまで説明したように、`new`, `delete`または`malloc`, `free`による動的メモリ管理は、`shared_ptr`と比べて以下の問題があります。

* ポインタの参照先が普通の変数なのか、動的メモリ管理で作られたものなのかの判断が難しい
* 削除を忘れると「メモリ・リーク」が発生する
* 削除したオブジェクトを読み書きすると「実行時エラー」や「論理エラー」が発生する
* `vector`と組み合わせる場合、要素を削除する前に`delete`が必要になる
* 要素の削除を伴う汎用アルゴリズムが使えない

多くのプログラム書いて経験を積むまでは、これらは大した問題ではないように思えるかもしれません。<br>
それでも、特に理由がないかぎり、動的メモリ管理には`shared_ptr`クラスと`make_shared`関数を使うことをおすすめします。

>**【配列を管理するshared_ptr】**<br>
>```cpp
>shared_ptr<int[]> p = make_shared<int[]>(5);
>```
>のように、`<>`の中に`型名[]`と書くことで、`shared_ptr`でも配列を管理できます。<br>
>しかし、配列が必要なら`vector`を使えばいいので、この書き方が使われることは滅多にありません。


----

## 4 練習問題

----

以下の手順にしたがって、各問題のプログラムを完成させなさい。

1. `%%writefile ...`の下の行からがプログラムです。問題文に従ってプログラムを修正、または追加してください。
2. プログラムを修正したら、セルの右側にある`▶`をクリックします。すると、ファイルが保存されます。
3. 「動作テスト」セルの`▶`をクリックすると、2で保存したファイルがコンパイル＆実行され、実行結果が表示されます。<br>
   このセルは、修正したプログラムの動作を確認するために使ってください。
4. 「実行」セルの`▶`をクリックすると、2で保存したファイルがコンパイル＆実行され、結果の成否が判定されます。
5. 判定に成功したら`AC`と表示されます。次の問題に進んでください。
6. 失敗したら`WA`と表示されます(その前にエラーメッセージが表示される場合もあります)。<br>
   これは、プログラムのどこかにエラーがあることを意味します。<br>
   「動作テスト」を使ってエラーを修正し、`AC`を目指してください。


### ❓️問題１ 自動的なメモリ管理

自動的なメモリ管理によって、変数が削除される順番を確認したいです。<br>
数字が`1`, `2`, `3`, `4`の順番で出力されるように、変数`a`, `b`, `c`, `d`のコンストラクタ引数に`1`～`4`のいずれかの数字を入れなさい。

**出力例**

```txt
1
2
3
4
```


In [ ]:
%%writefile practice_01a.cpp
#include <iostream>
using namespace std;

struct S {
  S(int n) : x(n) {} // コンストラクタ
  ~S() { cout << x << endl; } // デストラクタ

  int x = 0;
};

int main() {

  // 以下の4つの？を、1～4のいずれかの数字に書き換える
  S a(？);
  {
    S b(？);
    {
      S c(？);
    }
    S d(？);
  }
}

In [ ]:
# @title 動作テスト
!g++ -std=c++20 -O2 -Wall -Wextra -o practice_01a practice_01a.cpp && ./practice_01a

In [ ]:
# @title 実行
!diff -Z <(echo -e "1\n2\n3\n4") <(g++ -std=c++20 -O2 -Wall -Wextra -o practice_01a practice_01a.cpp && ./practice_01a) > /dev/null && test $? -eq 0 && echo -e "\033[32;1mAC" || echo -e "\033[31;1mWA"

In [ ]:
# @title 🔒解答例(どうしても問題を解けない場合に見てください)
%%writefile practice_01a.cpp
#include <iostream>
using namespace std;

struct S {
  S(int n) : x(n) {} // コンストラクタ
  ~S() { cout << x << endl; } // デストラクタ

  int x = 0;
};

int main() {

  // 以下の4つの？を、1～4のいずれかの数字に書き換える
  S a(4);
  {
    S b(3);
    {
      S c(1);
    }
    S d(2);
  }
}

### ❓️問題２ 静的なメモリ管理

自動的なメモリ管理に加えて、静的なメモリ管理によって、変数が削除される順番を確認したいです。<br>
数字が`1`, `2`, `3`, `4`, `5`の順番で出力されるように、変数`a`, `b`, `c`, `d`, `e`のコンストラクタ引数に`1`～`5`のいずれかの数字を入れなさい。

**出力例**

```txt
1
2
3
4
5
```


In [ ]:
%%writefile practice_01b.cpp
#include <iostream>
using namespace std;

struct S {
  S(int n) : x(n) {} // コンストラクタ
  ~S() { cout << x << endl; } // デストラクタ

  int x = 0;
};

// 以下の5つの？を、1～5のいずれかの数字に書き換える
S e(？);

int main() {

  S a(？);
  {
    S b(？);
    {
      static S c(？);
    }
    static S d(？);
  }
}

In [ ]:
# @title 動作テスト
!g++ -std=c++20 -O2 -Wall -Wextra -o practice_01b practice_01b.cpp && ./practice_01b

In [ ]:
# @title 実行
!diff -Z <(echo -e "1\n2\n3\n4\n5") <(g++ -std=c++20 -O2 -Wall -Wextra -o practice_01b practice_01b.cpp && ./practice_01b) > /dev/null && test $? -eq 0 && echo -e "\033[32;1mAC" || echo -e "\033[31;1mWA"

In [ ]:
# @title 🔒解答例(どうしても問題を解けない場合に見てください)
%%writefile practice_01b.cpp
#include <iostream>
using namespace std;

struct S {
  S(int n) : x(n) {} // コンストラクタ
  ~S() { cout << x << endl; } // デストラクタ

  int x = 0;
};

// 以下の5つの？を、1～5のいずれかの数字に書き換える
S e(5);

int main() {

  S a(2);
  {
    S b(1);
    {
      static S c(4);
    }
    static S d(3);
  }
}

### ❓️問題３ 動的なメモリ管理

動的なメモリ管理の動作を確認したいです。以下の処理を行うプログラムを完成させなさい。

1. `new`キーワードを使って、`int`型のオブジェクトを動的に作成する
2. 標準入力`cin`から、1で作成したオブジェクトに整数を読み込む
3. 標準出力`cout`に、1で作成したオブジェクトの値を出力し、改行する
4. `delete`キーワードを使って、1で作成したオブジェクトを解放する

**入力データ例（１）**

```txt
123
```

**出力例（１）**

```txt
123
```

**入力データ例（２）**

```txt
87654321
```

**出力例（２）**

```txt
87654321
```


In [ ]:
%%writefile practice_02a.cpp
#include <iostream>
using namespace std;

int main() {
  // この下に、1～4を行うプログラムを書く

}

In [ ]:
# @title 動作テスト
!g++ -std=c++20 -O2 -Wall -Wextra -o practice_02a practice_02a.cpp && echo "この下をクリックして、整数を入力" && ./practice_02a

In [ ]:
# @title 実行
!diff -Z <(echo -e "123\n87654321\n-57") <(g++ -std=c++20 -O2 -Wall -Wextra -o practice_02a practice_02a.cpp && echo "123" | ./practice_02a && echo "87654321" | ./practice_02a && echo "-57" | ./practice_02a) > /dev/null && test $? -eq 0 && echo -e "\033[32;1mAC" || echo -e "\033[31;1mWA"

In [ ]:
# @title 🔒解答例(どうしても問題を解けない場合に見てください)
%%writefile practice_02a.cpp
#include <iostream>
using namespace std;

int main() {
  // この下に、1～4を行うプログラムを書く
  int* p = new int;

  cin >> *p;
  cout << *p << endl;

  delete p;
}

### ❓️問題４ 配列を動的に作成する

動的なメモリ管理によって、配列を作成する方法を確認したいです。以下の処理を行うプログラムを完成させなさい。

1. `int`型の変数`n`を宣言し、`cin`から`n`にデータ数を読み込む
2. `new`を使って、長さ`n`の`int`型の配列を動的に作成する
3. for文を使って、次の処理を`n`回実行する
    1. 標準入力`cin`から、2で作成した配列の`i`番目に整数を読み込む
4. `reverse`関数を使って、2で作成した配列の順序を逆にする
5. for文を使って、次の処理を`n`回実行する
    1. 標準出力`cout`に、2で作成した配列の`i`番目の値を出力する
    2. `cout`に1個の空白を出力する
6. `cout`に開業を出力する
7. `delete[]`を使って、2で作成した配列を解放する

**入力データ例（１）**

```txt
5
1 4 1 4 2
```

**出力例（１）**

```txt
2 4 1 4 1
```

**入力データ例（２）**

```txt
1
57
```

**出力例（２）**

```txt
57
```


In [ ]:
%%writefile practice_02b.cpp
#include <iostream>
#include <algorithm>
using namespace std;

int main() {
  // この下に、1～7を行うプログラムを書く

}

In [ ]:
# @title 動作テスト
!g++ -std=c++20 -O2 -Wall -Wextra -o practice_02b practice_02b.cpp && echo "この下をクリックして、整数の数と、整数のリストを入力" && ./practice_02b

In [ ]:
# @title 実行
!diff -Z <(echo -e "2 4 1 4 1\n57\n9 8 7 6 5 4 3 2 1") <(g++ -std=c++20 -O2 -Wall -Wextra -o practice_02b practice_02b.cpp && echo "5 1 4 1 4 2" | ./practice_02b && echo "1 57" | ./practice_02b && echo "9 1 2 3 4 5 6 7 8 9" | ./practice_02b) > /dev/null && test $? -eq 0 && echo -e "\033[32;1mAC" || echo -e "\033[31;1mWA"

In [ ]:
# @title 🔒解答例(どうしても問題を解けない場合に見てください)
%%writefile practice_02b.cpp
#include <iostream>
#include <algorithm>
using namespace std;

int main() {
  // この下に、1～7を行うプログラムを書く
  int n;
  cin >> n;

  int* p = new int[n];

  for (int i = 0; i < n; i++) {
    cin >> p[i];
  }

  reverse(p, p + n);

  for (int i = 0; i < n; i++) {
    cout << p[i] << ' ';
  }
  cout << endl;

  delete[] p;
}

### ❓️問題５ シェアード・ポインタ

`shared_ptr`型の動作を確認したいです。以下の処理を行うプログラムを完成させなさい。

1. `make_shared`関数を使って、`int`型のオブジェクトを動的に作成する
2. 標準入力`cin`から、1で作成したオブジェクトに整数を読み込む
3. 標準出力`cout`に、1で作成したオブジェクトの値を出力し、改行する

**入力データ例（１）**

```txt
54321
```

**出力例（１）**

```txt
54321
```

**入力データ例（２）**

```txt
-57
```

**出力例（２）**

```txt
-57
```


In [ ]:
%%writefile practice_3a.cpp
#include <iostream>
#include <memory>
using namespace std;

int main() {
  // この下に、1～3を行うプログラムを書く

}

In [ ]:
# @title 動作テスト
!g++ -std=c++20 -O2 -Wall -Wextra -o practice_03a practice_03a.cpp && echo "この下をクリックして、整数を入力" && ./practice_03a

In [ ]:
# @title 実行
!diff -Z <(echo -e "1\n87654321\n-57") <(g++ -std=c++20 -O2 -Wall -Wextra -o practice_03a practice_03a.cpp && echo "1" | ./practice_03a && echo "87654321" | ./practice_03a && echo "-57" | ./practice_03a) > /dev/null && test $? -eq 0 && echo -e "\033[32;1mAC" || echo -e "\033[31;1mWA"

In [ ]:
# @title 🔒解答例(どうしても問題を解けない場合に見てください)
%%writefile practice_03a.cpp
#include <iostream>
#include <memory>
using namespace std;

int main() {
  // この下に、1～3を行うプログラムを書く
  auto p = make_shared<int>();

  cin >> *p;
  cout << *p << endl;
}

### ❓️問題６ XとYの戦い

体力と攻撃力を持つ2体のモンスターXとYが戦っています。<br>
次の手順に従って、戦いで勝ったほうの記号を出力するプログラムを完成させなさい。

>1. `int`型の変数`a`と`b`を宣言し、`cin`から`a`と`b`にモンスターXの体力と攻撃力を読み込む
>2. ポインタ変数`x`を宣言し、`make_shared`関数によって作成した`Monster`型オブジェクトのポインタで初期化する<br>
>引数には`a`と`b`を渡す
>3. `int`型の変数`a`と`b`を宣言し、`cin`から`a`と`b`にモンスターXの体力と攻撃力を読み込む
>4. ポインタ変数`y`を宣言し、`make_shared`関数によって作成した`Monster`型オブジェクトのポインタで初期化する<br>
>引数には`c`と`d`を渡す
>5. for文を使って、次の処理を無限に繰り返す
>    1. `x`のメンバ関数`Attack`を使って`y`を攻撃する
>    2. if文を使って、`y`のメンバ関数`IsDead`が`true`を返したら次の処理を実行する
>        1. `cout`に文字`X`を出力し、改行する
>        2. `break`でループを終了する
>    3. `y`のメンバ関数`Attack`を使って`x`を攻撃する
>    4. if文を使って、`x`のメンバ関数`IsDead`が`true`を返したら次の処理を実行する
>        1. `cout`に文字`Y`を出力し、改行する
>        2. `break`でループを終了する

**入力データ例（１）**

```txt
10 3
8 4
```

**出力例（１）**

```txt
X
```

**入力データ例（２）**

```txt
100 26
200 15
```

**出力例（２）**

```txt
Y
```


In [ ]:
%%writefile practice_3b.cpp
#include <iostream>
#include <memory>
using namespace std;

// モンスター構造体
struct Monster {
public:
  Monster(int h, int p) : hp(h), power(p) {}
  ~Monster() = default;
  Monster(const Monster&) = default;
  Monster& operator=(const Monster&) = default;

  // 死んでいるならtrue
  bool IsDead() const { return hp <= 0; }

  // bに攻撃
  void Attack(shared_ptr<Monster> b) { b->hp -= power; }

private:
  int hp;      // 体力
  int power;   // 攻撃力
};

int main() {
  // --- ここから上は変更しない ---

  // この下に、2体のモンスターが戦うプログラムを書く

}

In [ ]:
# @title 動作テスト
!g++ -std=c++20 -O2 -Wall -Wextra -o practice_03b practice_03b.cpp && echo "この下をクリックして、2体のモンスターのパラメータを入力" && ./practice_03b

In [ ]:
# @title 実行
!diff -Z <(echo -e "X\nY\nY") <(g++ -std=c++20 -O2 -Wall -Wextra -o practice_03b practice_03b.cpp && echo "10 3 8 4" | ./practice_03b && echo "100 26 200 15" | ./practice_03b && echo "2 2 3 2" | ./practice_03b) > /dev/null && test $? -eq 0 && echo -e "\033[32;1mAC" || echo -e "\033[31;1mWA"

In [ ]:
# @title 🔒解答例(どうしても問題を解けない場合に見てください)
%%writefile practice_03b.cpp
#include <iostream>
#include <memory>
using namespace std;

// モンスター構造体
struct Monster {
public:
  Monster(int h, int p) : hp(h), power(p) {}
  ~Monster() = default;
  Monster(const Monster&) = default;
  Monster& operator=(const Monster&) = default;

  // 死んでいるならtrue
  bool IsDead() const { return hp <= 0; }

  // bに攻撃
  void Attack(shared_ptr<Monster> b) { b->hp -= power; }

private:
  int hp;      // 体力
  int power;   // 攻撃力
};

int main() {
  // --- ここから上は変更しない ---

  // この下に、2体のモンスターが戦うプログラムを書く
  int a, b;
  cin >> a >> b;
  auto x = make_shared<Monster>(a, b);

  int c, d;
  cin >> c >> d;
  auto y = make_shared<Monster>(c, d);

  for (;;) {
    x->Attack(y);
    if (y->IsDead()) {
      cout << "X" << endl;
      break;
    }
    y->Attack(x);
    if (x->IsDead()) {
      cout << "Y" << endl;
      break;
    }
  }
}

### ❓️問題７ 機械兵士の部品

機械兵士を扱うプログラムを作っています。機械兵士には以下の3つの機能を追加できます。

* `MoveComponent`(ムーブ・コンポーネント): 移動
* `ShootComponent`(シュート・コンポーネント): 射撃
* `MeleeComponent`(メレー・コンポーネント): 近接攻撃

各機能には実行間隔(じっこうかんかく)が設定されます。例えば、実行間隔が2の場合、その機能は2分ごとに実行されます。<br>

移動の実行間隔A, 射撃の実行間隔B, 近接攻撃の実行間隔C、総実行時間Tが入力されます。<br>
実行間隔A, B, Cから機械兵士オブジェクトを初期化し、T分のあいだ行動を実行させるプログラムを作成しなさい。

* 実行間隔が`1`以上の場合: 対応する機能を追加し、実行間隔を設定する
* 実行間隔が`0`の場合: 対応する機能は追加しない。
* 機械兵士の行動時間: `1` ～ `T`
* 機械兵士に行動を実行させるには: `Mech`(メック)構造体の`Do`(ドゥ)メンバ関数を呼び出す

なお、機械兵士の構造体と機能の構造体は完成しており、変更の必要はありません。

**入力データ形式**

```txt
A B C
T
```

<br><details><summary>🗝️(クリックでヒントを見る)</summary>

**プログラム例**

1. `int`型の変数`a`, `b`, `c`を宣言する
2. `cin`から変数`a`, `b`, `c`に、実行間隔を読み込む
3. `int`型の変数`t`を宣言し、`cin`から`t`に総実行時間を読み込む
4. `Mech`型の変数`mech`を宣言する
5. if文を使って、変数`a`が0より大きければ次の処理を行う
    1. `make_shared`関数を使って、`MoveComponent`オブジェクトの引数に`a`を指定して作成し、`mech.move`に代入
6. if文を使って、変数`b`が0より大きければ次の処理を行う
    1. `make_shared`関数を使って、`ShootComponent`オブジェクトの引数に`b`を指定して作成し、`mech.shoot`に代入
7. if文を使って、変数`c`が0より大きければ次の処理を行う
    1. `make_shared`関数を使って、`MeleeComponent`オブジェクトの引数に`c`を指定して作成し、`mech.melee`に代入
8. for文を使って、ループ変数`i`を`1`から`t`まで`1`ずつ増やしながら、次の処理を行う
    1. 変数`mech`の`Do`メンバ関数を、引数に`i`を指定して呼び出す

</details><br>

**入力データ例（１）**

```txt
1 3 2
6
```

**出力例（１）**

```txt
move move melee move shoot move melee move move shoot melee
```

**入力データ例（２）**

```txt
0 2 3
5
```

**出力例（２）**

```txt
shoot melee shoot
```


In [ ]:
%%writefile practice_03c.cpp
#include <iostream>
#include <vector>
#include <memory>
#include <algorithm>
using namespace std;

// 移動機能の部品
struct MoveComponent {
  // コンストラクタ
  //  引数 t : 実行間隔
  explicit MoveComponent(int t) : t(t) {}

  void Do() { cout << "move "; }

  int t;
};

// 射撃機能の部品
struct ShootComponent {
  // コンストラクタ
  //  引数 t : 実行間隔
  explicit ShootComponent(int t) : t(t) {}

  void Do() { cout << "shoot "; }

  int t;
};

// 近接攻撃の部品
struct MeleeComponent {
  // コンストラクタ
  //  引数 t : 実行間隔
  explicit MeleeComponent(int t) : t(t) {}

  void Do() { cout << "melee "; }

  int t;
};

// 機械兵士
struct Mech {
  ~Mech() { cout << endl; }

  // 行動を実行
  //   引数 d : 経過時間
  void Do(int d) {
    if (move && d % move->t == 0) {
      move->Do();
    }
    if (shoot && d % shoot->t == 0) {
      shoot->Do();
    }
    if (melee && d % melee->t == 0) {
      melee->Do();
    }
  }

  shared_ptr<MoveComponent> move;   // 移動機能
  shared_ptr<ShootComponent> shoot; // 射撃機能
  shared_ptr<MeleeComponent> melee; // 近接攻撃機能
};

int main() {
  // --- ここから上は変更しない ---

  // この下に、機械兵士を作成し、機械兵士の行動を実行するプログラムを書く

}

In [ ]:
# @title 動作テスト
!g++ -std=c++20 -O2 -Wall -Wextra -o practice_03c practice_03c.cpp && echo "この下をクリックして、行動リストと行動時間を入力" && ./practice_03c

In [ ]:
# @title 実行
!diff -Z <(echo -e "shoot melee shoot\n\nmove move melee move shoot move melee move move shoot melee\nmove shoot melee") <(g++ -std=c++20 -O2 -Wall -Wextra -o practice_03c practice_03c.cpp && echo "0 2 3 5" | ./practice_03c && echo "0 0 0 100" | ./practice_03c && echo "1 3 2 6" | ./practice_03c && echo "5 5 5 5" | ./practice_03c) > /dev/null && test $? -eq 0 && echo -e "\033[32;1mAC" || echo -e "\033[31;1mWA"

In [ ]:
# @title 🔒解答例(どうしても問題を解けない場合に見てください)
%%writefile practice_03c.cpp
#include <iostream>
#include <vector>
#include <memory>
#include <algorithm>
using namespace std;

// 移動機能の部品
struct MoveComponent {
  // コンストラクタ
  //  引数 t : 実行間隔
  explicit MoveComponent(int t) : t(t) {}

  void Do() { cout << "move "; }

  int t;
};

// 射撃機能の部品
struct ShootComponent {
  // コンストラクタ
  //  引数 t : 実行間隔
  explicit ShootComponent(int t) : t(t) {}

  void Do() { cout << "shoot "; }

  int t;
};

// 近接攻撃の部品
struct MeleeComponent {
  // コンストラクタ
  //  引数 t : 実行間隔
  explicit MeleeComponent(int t) : t(t) {}

  void Do() { cout << "melee "; }

  int t;
};

// 機械兵士
struct Mech {
  ~Mech() { cout << endl; }

  // 行動を実行
  //   引数 d : 経過時間
  void Do(int d) {
    if (move && d % move->t == 0) {
      move->Do();
    }
    if (shoot && d % shoot->t == 0) {
      shoot->Do();
    }
    if (melee && d % melee->t == 0) {
      melee->Do();
    }
  }

  shared_ptr<MoveComponent> move;   // 移動機能
  shared_ptr<ShootComponent> shoot; // 射撃機能
  shared_ptr<MeleeComponent> melee; // 近接攻撃機能
};

int main() {
  // --- ここから上は変更しない ---

  // この下に、機械兵士を作成し、機械兵士の行動を実行するプログラムを書く
  int a, b, c, t;
  cin >> a >> b >> c >> t;

  Mech mech;
  if (a > 0) {
    mech.move = make_shared<MoveComponent>(a);
  }
  if (b > 0) {
      mech.shoot = make_shared<ShootComponent>(b);
  }
  if (c > 0) {
    mech.melee = make_shared<MeleeComponent>(c);
  }

  for (int i = 1; i <= t; i++) {
    mech.Do(i);
  }
}

### ❓️問題８ ブロック配置

`shared_ptr`を使って、平面マップにブロックを配置するプログラムを作りたいです。

最初、マップにはN個のブロックがあり、各ブロックのX座標は $ X_1 $ ～ $ X_N $ 、Y座標は $ Y_1 $ ～ $ Y_N $ であらわされます。<br>
次に、M個のブロックの追加または削除のコマンド $ C_1 $ ～ $ C_M $ が続きます。コマンドには次の２種類があります。

* `a X Y`: ブロック配列の末尾に、座標(X, Y)のブロックを追加
* `b Z`: ブロック配列のZ番目にあるブロックを削除

すべてのコマンドを処理したあと、残りの全ブロックの座標を出力するプログラムを作成しなさい。

**入力データ形式**

```txt
N
X1 Y1
X2 Y2
︙ ︙
XN YN
M
C1
C2
︙
CM
```

**出力形式**

各ブロックについて、記号`(`、ブロックのX座標、記号`,`、ブロックのY座標、記号`)`、空白1文字、の順で出力する。<br>
全てのブロックを出力したら、改行を出力する。

```txt
(ブロック1のX座標,ブロック1のY座標) (ブロック2のX座標,ブロック2のY座標) ... 改行
```

<br><details><summary>🗝️(クリックでヒントを見る)</summary>

**プログラム例**

1. `int`型の変数`n`を宣言し、`cin`から`n`にブロックの個数を読み込む
2. `vector<shared_ptr<Block>>`型の配列変数`data`を宣言し、サイズ`n`で初期化する
3. for文を使って、次の処理を`n`回行う
    1. `auto`型の変数`p`を宣言し、`make_shared<Block>()`で初期化する
    2. `cin`から`p`に「ブロックの座標」を読み込む
    3. `p`を配列変数`data`の`i`番目に代入する
4. `int`型の変数`m`を宣言し、`cin`から`m`に「コマンド数」を読み込む
5. for文を使って、次の処理を`m`回行う
    1. `char`型の変数`c`を宣言し、`cin`から`c`に「コマンドの最初の文字」を読み込む
    2. if文を使って、変数`c`が文字`'a'`と等しい場合、次の処理を行う
        1. `auto`型の変数`p`を宣言し、`make_shared<Block>()`で初期化する
        2. `cin`から`p`に「ブロックの座標」を読み込む
        3. `push_back`関数を使って、`p`を配列変数`data`二追加する
    3. else句を使って、次の処理を行う
        1. `int`型の変数`a`を宣言し、`cin`から`a`に「削除するデータの位置」を読み込む
        2. `erase`関数を使って、配列変数`data`から`a`番目の要素を削除する
6. for文とイテレータを使って、配列変数`data`の各要素に次の処理を行う
    1. `cout`に、記号`(`を出力する
    2. `cout`に、イテレータが指す要素のX座標を出力する
    3. `cout`に、記号`,`を出力する
    4. `cout`に、イテレータが指す要素のY座標を出力する
    5. `cout`に、記号`)`、空白1文字の順で出力する
7. `cout`に改行を出力する

</details><br>

**入力データ例（１）**

```txt
3
1 2
5 6
7 3
2
b 1
a 4 4
```

**出力例（１）**

```txt
(1,2) (7,3) (4,4)
```

**入力データ例（２）**

```txt
1
12 21
1
b 0
```

**出力例（２）**

```txt

```

ブロックが0個になった場合は、改行のみを出力します。


In [ ]:
%%writefile practice_04a.cpp
#include <iostream>
#include <vector>
#include <memory>
#include <algorithm>
using namespace std;

// ブロック構造体
struct Block {
  int x, y;   // ブロックの座標
};

int main() {
  // この下に、ブロックの配置データを読み込み、配置を更新した結果を出力するプログラムを書く

}

In [ ]:
# @title 動作テスト
!g++ -std=c++20 -O2 -Wall -Wextra -o practice_04a practice_04a.cpp && echo "この下をクリックして、ブロックのデータを入力" && ./practice_04a

In [ ]:
# @title 実行
!diff -Z <(echo -e "(1,2) (7,3) (4,4)\n\n(4,8) (3,3) (4,6) (0,1) (0,2) (0,3) ") <(g++ -std=c++20 -O2 -Wall -Wextra -o practice_04a practice_04a.cpp && echo "3 1 2 5 6 7 3 2 b 1 a 4 4" | ./practice_04a && echo "1 12 21 1 b 0" | ./practice_04a && echo "5 4 8 2 2 3 3 4 6 0 1 3 a 0 2 a 0 3 b 1" | ./practice_04a) > /dev/null && test $? -eq 0 && echo -e "\033[32;1mAC" || echo -e "\033[31;1mWA"

In [ ]:
# @title 🔒解答例(どうしても問題を解けない場合に見てください)
%%writefile practice_04a.cpp
#include <iostream>
#include <vector>
#include <memory>
#include <algorithm>
using namespace std;

// ブロック構造体
struct Block {
  int x, y;   // ブロックの座標
};

int main() {
  // この下に、ブロックの配置データを読み込み、配置を更新した結果を出力するプログラムを書く
  int n;
  cin >> n;

  vector<shared_ptr<Block>> data(n);
  for (int i = 0; i < n; i++) {
    data[i] = make_shared<Block>();
    cin >> data[i]->x >> data[i]->y;
  }

  int m;
  cin >> m;
  for (int i = 0; i < m; i++) {
    char c;
    cin >> c;
    if (c == 'a') {
      auto p = make_shared<Block>();
      cin >> p->x >> p->y;
      data.push_back(p);
    } else {
      int a;
      cin >> a;
      data.erase(data.begin() + a);
    }
  }

  for (auto i = data.begin(); i != data.end(); i++) {
    cout << "(" << (*i)->x << "," << (*i)->y << ") ";
  }
  cout << endl;
}